In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load MNIST dataset
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize to [-1, 1]

In [3]:
# Set up discriminator
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(28, 28, 1)),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

discriminator = make_discriminator_model()
discriminator.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy')

In [4]:
# Set up generator
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(7*7*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((7, 7, 256)),
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

generator = make_generator_model()

In [5]:
# Define the GAN
def make_gan(generator, discriminator):
    discriminator.trainable = False
    gan = tf.keras.Sequential([generator, discriminator])
    gan.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy')
    return gan

gan = make_gan(generator, discriminator)

In [6]:
# Training loop
def train_gan(gan, generator, discriminator, dataset, epochs=5 , batch_size=128):
    for epoch in range(epochs):
        for batch in dataset:
            # Generate noise
            noise = tf.random.normal([batch_size, 100])

            # Generate fake images
            generated_images = generator(noise)

            # Combine real and fake images
            combined_images = tf.concat([generated_images, batch], axis=0)

            # Labels for real and fake images
            labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0)

            # Add noise to labels
            labels += 0.05 * tf.random.uniform(tf.shape(labels))

            # Train discriminator
            d_loss = discriminator.train_on_batch(combined_images, labels)

            # Train generator
            noise = tf.random.normal([batch_size, 100])
            misleading_labels = tf.zeros((batch_size, 1))
            g_loss = gan.train_on_batch(noise, misleading_labels)

        # Print progress
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

# Convert training images to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(60000).batch(128)

# Train the GAN
train_gan(gan, generator, discriminator, train_dataset)


ValueError: Data cardinality is ambiguous:
  x sizes: 224
  y sizes: 256
Make sure all arrays contain the same number of samples.

: 